In [ ]:
# import warnings
import time

# For Data science
import spacy
import pandas as pd
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer

import matplotlib.pyplot as plt

from sklearn import (
    metrics,
    linear_model,
)

from sources import three_sigma_cleared

# Getting data, observations
## Get dataset

In [ ]:
# Get dataset from file
df = pd.read_csv(
    f"../data/IMDB_Dataset.csv",
)

# Show dataset head
df.head()

In [ ]:
review = df.columns[-1]

grouped = df.groupby(review).size()

In [ ]:
grouped.plot(
    kind='bar',
    xlabel='Review',
    ylabel='Count',
    title='Count of Reviews',
);

Dataframe is balanced.

In [ ]:
# Set feature and target subsets
df['word_count'] = df['review'].apply(lambda x: len(x.split()))

# Plot the distribution of word counts
plt.figure(figsize=(10, 6))
plt.title("Distribution of Word Counts")
sns.histplot(x="word_count", data=df, kde=True)
plt.xlabel("Number of Words")
plt.ylabel("Frequency")
plt.show()

In [ ]:
df2 = three_sigma_cleared(
    dataset=df,
    feature_names=['word_count'],
)

In [ ]:
plt.figure(figsize=(10, 6))
sns.boxplot(data=df, x='sentiment', y='word_count')
plt.title('Box Plot of Sentiment vs Word Count')
plt.xlabel('Sentiment')
plt.ylabel('Word Count');

# Preprocessing

In [ ]:
# Load the English language model with all the pipeline components
nlp = spacy.load("en_core_web_sm")

In [ ]:
list(nlp.pipe_labels.keys())

In [ ]:
nlp.factory_names

In [ ]:
nlp.pipe_names

# Lasy use

In [ ]:
# Load the English model
nlp = spacy.load(
    name="en_core_web_sm",
    # disable=[
    #     "tagger",
    #     "attribute_ruler",
    #     "lemmatizer"
    # ],
)

nlp.vocab.reset_vectors(width=64)

# Preprocess text function
def spacy_preprocessed_text(text):
    doc = nlp(text)
    tokens = [token.lemma_ for token in doc if not token.is_stop and not token.is_punct]
    
    return ' '.join(tokens)

start_time = time.time()

df = df[:5000]

df['processed_review'] = df['review'].apply(spacy_preprocessed_text)

print("--- %s seconds ---" % (time.time() - start_time))

In [ ]:
df['sentiment'] = df['sentiment'].map({'positive': 1, 'negative': 0})

In [ ]:
train_df, test_df = train_test_split(
    df,
    test_size=0.3,
    random_state=42,
)

In [ ]:
train_df

In [ ]:
vectorizer = CountVectorizer()

X_train = vectorizer.fit_transform(train_df['processed_review'])
X_test = vectorizer.transform(test_df['processed_review'])
y_train = train_df['sentiment']
y_test = test_df['sentiment']

In [ ]:
classifier = linear_model.LogisticRegression(
    max_iter=500,
)

classifier.fit(X_train, y_train)

prediction = classifier.predict(X_test)

In [ ]:
confusion_matrix = metrics.confusion_matrix(
    y_true=y_test,
    y_pred=prediction,
    labels=classifier.classes_,
)

confusion_matrix

In [ ]:
disp = metrics.ConfusionMatrixDisplay(
    confusion_matrix=confusion_matrix,
    display_labels=classifier.classes_,
)

disp.plot(cmap='Blues');

In [ ]:
# Get density plot
# for test data
sns.kdeplot(
    y_test,
    fill=False,
    color='r',
    label='test subset',
)

# for predicted data
sns.kdeplot(
    prediction,
    fill=True,
    color='b',
    label='predicted',
)

# Plot
plt.title('Distribution of observations in test dataset and and predicted dataset')
plt.legend();

In [ ]:
print(
    metrics.classification_report(
    y_true=y_test,
    y_pred=prediction,
    target_names=['class 1', 'class 2'],
    )
)